In [13]:

RENDER_DB_HOST = "dpg-d4mbue3uibrs738g22qg-a.oregon-postgres.render.com"
RENDER_DB_PORT = "5432"
RENDER_DB_NAME = "normalized_db"
RENDER_DB_USER = "normalized_db_user"
RENDER_DB_PASS = "zeG2hoet9EG6pkSbTqpgOMXEu1dMnBEd" 

print("Credentials set locally for the notebook session.")

Credentials set locally for the notebook session.


In [27]:
import pandas as pd
from sujal_codio_project import create_connection, ex1, ex2, ex3, ex4, ex5, ex6, ex7, ex8, ex9, ex10, ex11
import psycopg2
from psycopg2 import Error

def create_pg_connection():
    """Establishes a connection to the Render PostgreSQL database."""
    conn = None
    try:
        conn = psycopg2.connect(
            dbname=RENDER_DB_NAME,
            user=RENDER_DB_USER,
            password=RENDER_DB_PASS,
            host=RENDER_DB_HOST,
            port=RENDER_DB_PORT
        )

        print("Connection to Render successful.")
        return conn

    except Error as e:
        print(f"PostgreSQL Connection Error: {e}")
        return None


def create_table(conn, create_table_sql, drop_table_name=None):
    """Creates a table, optionally dropping the old one."""
    if conn is None:
        print("Connection is None, skipping table creation.")
        return

    try:
        cursor = conn.cursor()
        if drop_table_name:
            
            cursor.execute(f'DROP TABLE IF EXISTS "{drop_table_name}" CASCADE;')
            print(f"Dropped table: {drop_table_name}")
        
        cursor.execute(create_table_sql)
        conn.commit() 
        print(f"Table created/updated.")
        
    except Error as e:
        print(f"Error executing SQL for table creation: {e}")
        conn.rollback()
    finally:
        cursor.close()

In [28]:
conn = create_pg_connection()
conn


Connection to Render successful.


<connection object at 0x000001CE1137D360; dsn: 'user=normalized_db_user password=xxx dbname=normalized_db host=dpg-d4mbue3uibrs738g22qg-a.oregon-postgres.render.com port=5432', closed: 0>

In [29]:
cursor = conn.cursor()
cursor.execute("SELECT NOW();")
result = cursor.fetchall()
cursor.close()

result

[(datetime.datetime(2025, 12, 1, 1, 37, 16, 648703, tzinfo=datetime.timezone.utc),)]

In [30]:

DB_PATH = r"C:\Users\sujal\EAS503-project-sujal\normalized.db"

conn = create_connection(DB_PATH)
conn

In [31]:
tables = pd.read_sql_query(
    "SELECT name FROM sqlite_master WHERE type='table' ORDER BY NAME",
    conn
)
tables
print(tables.head())

              name
0          Country
1         Customer
2      OrderDetail
3          Product
4  ProductCategory


In [35]:
sql = ex7(conn)
print(sql)

df = pd.read_sql_query(sql, conn)
df.head()



    WITH RankedCountries AS (
      SELECT 
        R.Region,
        Co.Country,
        ROUND(SUM(P.ProductUnitPrice * O.QuantityOrdered)) as CountryTotal,
        RANK() OVER (
          PARTITION BY R.Region 
          ORDER BY SUM(P.ProductUnitPrice * O.QuantityOrdered) DESC
        ) as CountryRank
      FROM OrderDetail O
      JOIN Customer C ON O.CustomerID = C.CustomerID
      JOIN Product P ON O.ProductID = P.ProductID
      JOIN Country Co ON C.CountryID = Co.CountryID
      JOIN Region R ON Co.RegionID = R.RegionID
      GROUP BY R.Region, Co.Country
    )
    SELECT Region, Country, CountryTotal, CountryRank AS CountryRegionalRank
    FROM RankedCountries
    WHERE CountryRank = 1
    ORDER BY Region ASC, CountryRegionalRank ASC
    


,Region,Country,CountryTotal,CountryRegionalRank
0,British Isles,UK,34560755.0,1
1,Central America,Mexico,25146864.0,1
2,Eastern Europe,Poland,5196640.0,1
3,North America,USA,65090437.0,1
4,Northern Europe,Denmark,10611365.0,1
